In [7]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [8]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [9]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


# Step 1 : Create source embeddings for the text column

In [10]:
from sentence_transformers import SentenceTransformer

In [11]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

c:\Users\Mihir\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   2%|2         | 10.5M/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
vectors.shape

(8, 768)

In [13]:
dim = vectors.shape[1]
dim

768

# Step 2 : Build a FAISS Index for vectors

In [14]:
import faiss

index = faiss.IndexFlatL2(dim)

# Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [15]:
index.add(vectors)

In [16]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000002793285D8F0> >

# Step 4 : Encode search text using same encorder and normalize the output vector

In [17]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [18]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

# Step 5: Search for similar vector in the FAISS index created

In [20]:
distances, I = index.search(svec, k=2)
distances

array([[1.3844838, 1.4039094]], dtype=float32)

In [21]:
I.tolist()

[[3, 2]]

In [22]:
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [23]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion
